# Preparation

## Library

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.sampler import WeightedRandomSampler
from torch.optim import lr_scheduler

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.inspection import permutation_importance

from fetchData import fetchdata, cv_events
import __MLP
# from __MLP import getSamplers, convert_df_to_unsqueezed_tensor, train_sequential, clf_report
import random

import __Preprocessing
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

pd.set_option('display.max_columns', None)

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [16]:
seed = 42
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)


torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

def _init_fn(worker_id):
    np.random.seed(int(seed))

def test_data_process(X_test, y_test):
    tensor_x1 = torch.Tensor(X_test.values).unsqueeze(1)
    tensor_y1 = torch.Tensor(y_test.values).unsqueeze(1)
    test_dataset = TensorDataset(tensor_x1,tensor_y1)

    batch_size = 8

    # train_sampler, test_sampler = __MLP.getSamplers(pheme_y, tensor_x2)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    data = next(iter(test_dataloader))
    print("mean: %s, std: %s" %(data[0].mean(), data[0].std()))

    test_size = int(tensor_y1.size(0))

    print("Test Size",test_size)

    # predict_batch
    return test_dataloader, test_size


def test_data_process(X_test, y_test):

    tensor_x1 = torch.Tensor(X_test.values).unsqueeze(1)
    tensor_y1 = torch.Tensor(y_test.values).unsqueeze(1)
    test_dataset = TensorDataset(tensor_x1,tensor_y1)

    batch_size = 8

    # train_sampler, test_sampler = __MLP.getSamplers(pheme_y, tensor_x2)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    data = next(iter(test_dataloader))
    print("mean: %s, std: %s" %(data[0].mean(), data[0].std()))

    test_size = int(tensor_y1.size(0))

    print("Test Size",test_size)

    # predict_batch
    return test_dataloader, test_size
def predict(model, criterion, val_dataloader, val_size):
    model.eval()
    val_label_list = []
    # val_preds_list = []
    running_val_preds = []

    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0
        f1_running = 0
        for j, val in enumerate(val_dataloader, 0):
            val_x, val_label = val
            val_x, val_label = val_x.float(), val_label.float()
            val_outputs = model(val_x)
            val_preds = val_outputs.squeeze(1) > 0.0
            f1_running += (f1_score(val_label, val_preds,zero_division=True) * val_x.size(0))
            v_loss = criterion(val_outputs, val_label.unsqueeze(1))
            val_loss += (v_loss.item() * val_x.size(0))
            val_corrects += torch.sum(val_preds == val_label)
            val_label_list.append(val_label)
            running_val_preds.append(val_preds)

    running_val_preds = torch.cat(running_val_preds, 0)
    val_label_list = torch.cat(val_label_list, 0)
    val_corrects = val_corrects
    val_loss = val_loss/val_size
    val_acc = val_corrects.double().numpy() / val_size
    f1_running /= val_size
    print("accuracy_score:\t\t%.4f" % val_acc)
    print('Precision Score:\t%.4f' % precision_score(val_label_list,running_val_preds))
    print('Recall Score:\t\t%.4f' % recall_score(val_label_list,running_val_preds))
    print("f1_score:\t\t%.4f" % f1_running)
    print("Test_loss:\t\t%.4f" % val_loss)


NameError: name 'random' is not defined

## Data

In [33]:
# Final
pheme_sparse_final = pd.read_csv('./data/_PHEME_sparse.csv')
pheme_y = pd.read_csv('./data/_PHEME_target.csv').target
pheme_pos_final = pd.read_csv('./data/_PHEME_postags.csv')
pheme_thread_final_avg = pd.read_csv('./data/_PHEME_thread_avg.csv')
pheme_thread_final_std = pd.read_csv('./data/_PHEME_thread_std.csv')

ext_pos_final = pd.read_csv('./data/_PHEMEext_postags.csv')
ext_sparse_final = pd.read_csv('./data/_PHEMEext_sparse.csv')
ext_y = pd.read_csv('./data/_PHEMEext_text.csv').target
ext_thread_final_avg = pd.read_csv('./data/_PHEMEext_thread_avg.csv')
ext_thread_final_std = pd.read_csv('./data/_PHEMEext_thread_std.csv')

pheme_bert_simple_normal = pd.read_csv('./data/_PHEME_Bert_final_brackets.csv')
pheme_bert_brackets_normal = pd.read_csv('./data/_PHEME_Bert_final_brackets_nrmzd.csv')
ext_bert_simple_normal = pd.read_csv('./data/_PHEMEext_Bert_final_brackets.csv')
ext_bert_brackets_normal = pd.read_csv('./data/_PHEMEext_Bert_final_brackets_nrmzd.csv')

pheme_event = pd.read_csv('./data/_PHEME_text.csv')['Event']
ext_event = pd.read_csv('./data/_PHEMEext_text.csv').Event
pheme_AVGw2v = pd.read_csv('./data/_PHEME_text_AVGw2v.csv').drop(['token'],axis=1)
ext_AVGw2v = pd.read_csv('./data/_PHEMEext_text_AVGw2v.csv').drop(['token'],axis=1)

In [34]:
print("pheme_sparse_final", pheme_sparse_final.shape)
print(pheme_sparse_final.columns, "\n")
print("pheme_pos_final", pheme_pos_final.shape)
x = zip(pheme_pos_final.columns.values, ['Interjection', 'Hashtag', 'coordinating conjunction', 'punctuation', 'at-mention', 'proper noun', 'adjective' 'determiner' 'other', 'nominal + verbal',
    'common noun', 'pronoun', 'pre/postposition/subordinating conjunction', 'adverb', 'nominal + possessive', 'verb particle', 'URL or email', 'verb incl. copula, auxiliaries',
 'existential there, predeterminers', 'proper noun + possessive', 'discourse marker'])
print(set(x), "\n")
print("pheme_thread_final", pheme_thread_final_avg.shape)
print(pheme_thread_final_avg.columns, "\n")
print("ext_bert_simple_normal", ext_bert_simple_normal.shape)
print(pheme_bert_simple_normal.columns, "\n")


pheme_sparse_final (5802, 28)
Index(['emoji_count', 'URLcount', 'has_media', 'Skepticism', 'MentionCount',
       'FirstPersonPronoun', 'SecondPersonPronoun', 'ThirdPersonPronoun',
       'test_auxiliary', 'test_tentat', 'test_certain', 'Numeral',
       'char_count', 'word_count', 'HashTag', 'has_question', 'has_exclaim',
       'has_period', 'capital_ratio', 'retweet_count', 'tweet_count',
       'listed_count', 'friends_count', 'follower_count', 'followers/friend',
       'favourites_count', 'account_age_days', 'verified'],
      dtype='object') 

pheme_pos_final (5802, 21)
{('#', 'Hashtag'), ('t', 'verb incl. copula, auxiliaries'), (',', 'punctuation'), ('!', 'Interjection'), ('a', 'adjectivedeterminerother'), ('n', 'pre/postposition/subordinating conjunction'), ('d', 'nominal + verbal'), ('^', 'proper noun'), ('o', 'adverb'), ('p', 'nominal + possessive'), ('l', 'pronoun'), ('&', 'coordinating conjunction'), ('@', 'at-mention'), ('u', 'existential there, predeterminers'), ('x', 'd

In [35]:
# cv = cv_events(pd.concat([pheme_sparse, pheme_y, pheme_event],axis=1))
# X = cv[3][1].drop(['target', 'Event'],axis=1)
# y = cv[3][1].target
# val_X = cv[3][0].drop(['target', 'Event'],axis=1)
# val_y = cv[3][0].target

In [36]:
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

seed = 42

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [37]:
def getDataSize(tensor_x1, tensor_y1, tensor_x2, tensor_y2):
    train_size = int(tensor_y1.size(0))
    test_size = int(tensor_y2.size(0))

    print("Variables)\n\tTrain:%s\n\tTest: %s"%(tensor_x1.size(),tensor_x2.size()))
    # print("\tTargets:%s \ %s"%(tensor_y1.size()[0],tensor_y2.size()[0]))
    print("Train Size",train_size,"Test Size",test_size)
    print()
    return train_size, test_size

In [38]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()


# Sparse

In [51]:
class sparse_model(nn.Module):
    def __init__(self):
        super(sparse_model, self).__init__()  # 1*20
        self.layers = nn.Sequential(
            nn.Linear(28, 8),
            nn.ELU(),
            # nn.Linear(12, 8),
            # nn.ELU(),
            nn.Linear(8, 1)
        )
        self.drop_3 = nn.Dropout(0.3)
        self.drop_4 = nn.Dropout(0.4)
        self.drop_2 = nn.Dropout(0.2)

    def forward(self, x):
        x = self.layers(x)
        return x


In [52]:
cv_pd_list = cv_rumor(model_sparse, pd.concat([pheme_sparse_final,pheme_event],axis=1))

NameError: name 'cv_rumor' is not defined

In [53]:
len(train_dataloader)

233

In [178]:
def cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=20):
    # cv_pd_list[?][0]은 Training cv_pd_list[?][1] Testing
    cv_pd_list = []
    data = pd.concat([dataset, pheme_event, pheme_y], axis=1)
    NUM_EVENT = data.Event.unique().shape[0]
    EVENTS = data.Event.unique()
    results = {}

    for i, d in enumerate(EVENTS):
        df1, df2 = [x for _, x in data.groupby(data['Event'] != d)]
        df1.reset_index(inplace=True, drop=True)
        df2.reset_index(inplace=True, drop=True)
        cv_pd_list.append([df2, df1])


    for train, test in cv_pd_list:
        print("Train: %s \ Test: %s" % (train.shape, test.shape))

    print()

    for index, fold in enumerate(cv_pd_list):

        # DATA PREPARATION
        train, test = fold
        print("FOLD %d\n----------------------------------------------------------------------------" % (int(index)+1))
        train_target = train.pop('target')
        train.pop('Event')
        test_target = test.pop('target')
        test.pop('Event')

        # train, test = scaleData(train, test)

        tensor_x1, tensor_y1, tensor_x2, tensor_y2 = __MLP.convert_df_to_unsqueezed_tensor(
            train.values, train_target, test.values, test_target.values)
        train_dataset = TensorDataset(tensor_x1, tensor_y1)
        test_dataset = TensorDataset(tensor_x2, tensor_y2)

        batch_size = 16

        train_sampler, test_sampler = __MLP.getSamplers(train_target, tensor_x2)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                                    sampler=train_sampler, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                                    shuffle=False, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)

        data = next(iter(train_dataloader))
        print("mean: %s, std: %s" % (data[0].mean(), data[0].std()))

        train_size, test_size = getDataSize(
            tensor_x1, tensor_y1, tensor_x2, tensor_y2)

        model = sparse_model()

        # model_sparse = sparse_model()
        # criterion = nn.BCEWithLogitsLoss()
        # optimizer = optim.SGD(model_sparse.parameters(), lr=0.01, momentum=0.9)
        # optimizer = optim.Adam(model_sparse.parameters(), lr=5e-5, eps=1e-8, weight_decay=1e-7)
        optimizer = AdamW(model.parameters(),
                        # lr=5e-5,    # Default learning rate
                        lr=5e-5,    # Default learning rate
                        eps=1e-8,    # Default epsilon value
                        weight_decay=1e-6
                        )

        total_steps = len(train_dataloader) * epochs
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=0,  # Default value
                                                    num_training_steps=total_steps)

        PATH = "./Model/sparse_fold_"+str(index+1)+".pt"
        print(f'PATH: {PATH}\n')

        training_acc = []
        training_loss = []

        # train_acc, train_loss, val_acc, val_loss_list = __MLP.train_sequential(model=model, num_epochs=epochs, patience=patience, criterion=criterion, optimizer=optimizer, scheduler=scheduler, train_loader=train_dataloader, train_size=train_size, test_loader=test_dataloader, test_size=test_size, PATH=PATH)

        # Run the training loop for defined number of epochs
        for epoch in range(0, epochs):

            # Print epoch
            print(f'Starting epoch {epoch+1}')

            # Set current loss value
            current_loss = 0.0
            running_corrects = 0.0
            running_loss = 0.0

            # Iterate over the DataLoader for training data
            for i, data in enumerate(train_dataloader, 0):

                temp = []

                # Get inputs
                inputs, targets = data

                # Zero the gradients
                optimizer.zero_grad()

                # Perform forward pass
                outputs = model(inputs)

                # temp.append(outputs)
                # temp.append(inputs)
                # temp.append(targets)

                outputs = outputs.view(outputs.size(0), -1)

                # Compute Prediction Outputs
                # preds = outputs.squeeze(1) > 0.0
                preds = outputs > 0.0

                # Compute loss
                loss = criterion(outputs, targets)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == targets.data).data

                # Perform backward pass
                loss.backward()

                # Perform optimization and Scheduler
                optimizer.step()
                scheduler.step()

                # Print statistics
                # current_loss += loss.item() # 원본
                # if i % len(train_dataloader) == len(train_dataloader)-1:
                #     print('Loss after mini-batch %5d: %.3f' %
                #           (i + 1, current_loss / i+1))

                current_loss += loss.item() * inputs.size(0)
                if i % len(train_dataloader) == len(train_dataloader)-1:
                    print("Loss/ACC after mini-batch %5d: %.3f / %.4f" %
                        (i + 1, current_loss / train_size, running_corrects/train_size))

            # temp.append(running_corrects)
            # temp.append(running_corrects.double())
            # epoch_acc = running_corrects.double() / train_size
            epoch_acc = running_corrects / train_size
            epoch_loss = running_loss / train_size
            # temp.append(epoch_acc)
            training_acc.append(epoch_acc)
            training_loss.append(epoch_loss)
            # print('Epoch {}/{}\tTrain) Acc: {:.4f}, Loss: {:.4f}'.format(epoch+1,
                                                                        #  epochs, epoch_acc, epoch_loss))
        # Process is complete.
        print('Training process has finished. Saving trained model.')

        # Print about testing
        print('**Starting TESTING**')

        # Saving the model
        # save_path = f'./model-fold-{fold}.pth'
        torch.save(model.state_dict(), PATH)

        # Evaluation for this fold
        correct, total = 0, 0
        val_corrects=0
        with torch.no_grad():

            # Iterate over the test data and generate predictions
            for i, data in enumerate(test_dataloader, 0):

                # Get inputs
                inputs, targets = data

                # Generate outputs
                outputs = model(inputs)

                # Set total and correct
                _, predicted = torch.max(outputs.data, 1)

                outputs = outputs.view(outputs.size(0), -1)
                preds = outputs > 0.0
                # running_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == targets.data).data

                total += targets.size(0)
                correct += (predicted == targets).sum().item()

            # Print accuracy
            print('Accuracy for fold %d: %d %%' % (index, 100.0 * correct / total))
            print('F1 Score for fold %d: %d %%' %(index, f1_score(total, correct, zero_division=1)))
            print('Accuracy-2 for fold %d: %d %%' % (index, 100.0 * val_corrects / total))
            print('F1 Score-2 for fold %d: %d %%' %(index, f1_score(total, preds, zero_division=1)))
            print('F1 Score-3 for fold %d: %d %%' %(index, f1_score(total, predicted, zero_division=1)))
            print('--------------------------------')
            results[index] = 100.0 * (correct / total)


    # ---------------------------- Print fold results ---------------------------- #

    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {NUM_EVENT} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    print(f'Average: {sum/len(results.items())} %')


    return results

In [179]:
class sparse_model(nn.Module):
    def __init__(self):
        super(sparse_model, self).__init__()  # 1*20
        self.layers = nn.Sequential(
            nn.Linear(28, 8),
            nn.ELU(),
            # nn.Linear(12, 8),
            # nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 1)
        )
        self.drop_3 = nn.Dropout(0.3)
        self.drop_4 = nn.Dropout(0.4)
        self.drop_2 = nn.Dropout(0.2)

    def forward(self, x):
        x = self.layers(x)
        return x

model = sparse_model()
dataset = pheme_sparse_final
criterion = nn.BCEWithLogitsLoss()
epochs = 20

cv_process(dataset, criterion, events=pheme_event, target=pheme_y, epochs=epochs)

Train: (3723, 30) \ Test: (2079, 30)
Train: (4659, 30) \ Test: (1143, 30)
Train: (5333, 30) \ Test: (469, 30)
Train: (4912, 30) \ Test: (890, 30)
Train: (4581, 30) \ Test: (1221, 30)

FOLD 1
----------------------------------------------------------------------------
mean: tensor(15906.8613), std: tensor(161537.8438)
Variables)
	Train:torch.Size([3723, 1, 28])
	Test: torch.Size([2079, 1, 28])
Train Size 3723 Test Size 2079

PATH: ./Model/sparse_fold_1.pt

Starting epoch 1
Loss/ACC after mini-batch   233: 14849.725 / 0.5068
Starting epoch 2
Loss/ACC after mini-batch   233: 13195.729 / 0.4829
Starting epoch 3
Loss/ACC after mini-batch   233: 9991.574 / 0.5026
Starting epoch 4
Loss/ACC after mini-batch   233: 8821.296 / 0.4808
Starting epoch 5
Loss/ACC after mini-batch   233: 9043.665 / 0.4902
Starting epoch 6
Loss/ACC after mini-batch   233: 8830.099 / 0.4867
Starting epoch 7
Loss/ACC after mini-batch   233: 6816.245 / 0.5012
Starting epoch 8
Loss/ACC after mini-batch   233: 6156.383 / 0

TypeError: Expected sequence or array-like, got <class 'int'>

In [154]:
results

{}

In [158]:
# Class = {...}
# model = sparse_model()
dataset = pheme_sparse_final
# loss_function = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
# random.seed(42)
# torch.backends.cudnn.deterministic=True
# torch.backends.cudnn.benchmark = False

# ---------------------------------------------------------------------------- #
#                                INSIDE FUNCTION                               #
# ---------------------------------------------------------------------------- #

# cv_pd_list[?][0]은 Training cv_pd_list[?][1] Testing
cv_pd_list = []
data = pd.concat([dataset, pheme_event, pheme_y], axis=1)
# results['feature'] = nameof(dataset)
NUM_EVENT = data.Event.unique().shape[0]
EVENTS = data.Event.unique()
for i, d in enumerate(EVENTS):
    df1, df2 = [x for _, x in data.groupby(data['Event'] != d)]
    df1.reset_index(inplace=True, drop=True)
    df2.reset_index(inplace=True, drop=True)
    cv_pd_list.append([df2, df1])

for train, test in cv_pd_list:
    print("Train: %s \ Test: %s" % (train.shape, test.shape))

print()

for index, fold in enumerate(cv_pd_list):

    # DATA PREPARATION
    train, test = fold
    print("FOLD %d\n----------------------------------------------------------------------------" % (int(index)+1))
    train_target = train.pop('target')
    train.pop('Event')
    test_target = test.pop('target')
    test.pop('Event')

    # train, test = scaleData(train, test)

    tensor_x1, tensor_y1, tensor_x2, tensor_y2 = __MLP.convert_df_to_unsqueezed_tensor(
        train.values, train_target, test.values, test_target.values)
    train_dataset = TensorDataset(tensor_x1, tensor_y1)
    test_dataset = TensorDataset(tensor_x2, tensor_y2)

    batch_size = 16

    train_sampler, test_sampler = __MLP.getSamplers(train_target, tensor_x2)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                                  sampler=train_sampler, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                                 shuffle=False, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)

    data = next(iter(train_dataloader))
    print("mean: %s, std: %s" % (data[0].mean(), data[0].std()))

    train_size, test_size = getDataSize(
        tensor_x1, tensor_y1, tensor_x2, tensor_y2)

    model = sparse_model()

    epochs = 10
    patience = 20

    # model_sparse = sparse_model()
    # criterion = nn.BCEWithLogitsLoss()
    # optimizer = optim.SGD(model_sparse.parameters(), lr=0.01, momentum=0.9)
    # optimizer = optim.Adam(model_sparse.parameters(), lr=5e-5, eps=1e-8, weight_decay=1e-7)
    optimizer = AdamW(model.parameters(),
                      # lr=5e-5,    # Default learning rate
                      lr=5e-5,    # Default learning rate
                      eps=1e-8,    # Default epsilon value
                      weight_decay=1e-6
                      )

    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,  # Default value
                                                num_training_steps=total_steps)

    PATH = "./Model/sparse_fold_"+str(index+1)+".pt"
    print(f'PATH: {PATH}\n')

    training_acc = []
    training_loss = []

    # train_acc, train_loss, val_acc, val_loss_list = __MLP.train_sequential(model=model, num_epochs=epochs, patience=patience, criterion=criterion, optimizer=optimizer, scheduler=scheduler, train_loader=train_dataloader, train_size=train_size, test_loader=test_dataloader, test_size=test_size, PATH=PATH)

    # Run the training loop for defined number of epochs
    for epoch in range(0, epochs):
        # Print epoch
        print(f'Starting epoch {epoch+1}')

        # Set current loss value
        current_loss = 0.0
        running_corrects = 0.0
        running_loss = 0.0

        model.train()

        # Iterate over the DataLoader for training data
        for i, data in enumerate(train_dataloader, 0):

            temp = []

            # Get inputs
            inputs, targets = data

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # temp.append(outputs)
            # temp.append(inputs)
            # temp.append(targets)

            outputs = outputs.view(outputs.size(0), -1)

            # Compute Prediction Outputs
            # preds = outputs.squeeze(1) > 0.0
            preds = outputs > 0.0

            # Compute loss
            loss = criterion(outputs, targets)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == targets.data).data

            # Perform backward pass
            loss.backward()

            # Perform optimization and Scheduler
            optimizer.step()
            scheduler.step()

            # Print statistics
            # current_loss += loss.item() # 원본
            # if i % len(train_dataloader) == len(train_dataloader)-1:
            #     print('Loss after mini-batch %5d: %.3f' %
            #           (i + 1, current_loss / i+1))

            current_loss += loss.item() * inputs.size(0)
            if i % len(train_dataloader) == len(train_dataloader)-1:
                print("Loss/ACC after mini-batch %5d: %.3f / %.4f" %
                      (i + 1, current_loss / train_size, running_corrects/train_size))

        # temp.append(running_corrects)
        # temp.append(running_corrects.double())
        # epoch_acc = running_corrects.double() / train_size
        epoch_acc = running_corrects / train_size
        epoch_loss = running_loss / train_size
        # temp.append(epoch_acc)
        training_acc.append(epoch_acc)
        training_loss.append(epoch_loss)
        # print('Epoch {}/{}\tTrain) Acc: {:.4f}, Loss: {:.4f}'.format(epoch+1,
                                                                    #  epochs, epoch_acc, epoch_loss))

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('**Starting TESTING**')

    # Saving the model
    # save_path = f'./model-fold-{fold}.pth'
    torch.save(model.state_dict(), PATH)

    # Evaluation for this fold
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():

        # Iterate over the test data and generate predictions
        for i, data in enumerate(test_dataloader, 0):

            # Get inputs
            inputs, targets = data

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        # Print accuracy
        print('Accuracy for fold %d: %d %%' % (index, 100.0 * correct / total))
        print('--------------------------------')
        results[index] = 100.0 * (correct / total)

# ---------------------------- Print fold results ---------------------------- #

print(f'K-FOLD CROSS VALIDATION RESULTS FOR {NUM_EVENT} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average: {sum/len(results.items())} %')



Train: (3723, 30) \ Test: (2079, 30)
Train: (4659, 30) \ Test: (1143, 30)
Train: (5333, 30) \ Test: (469, 30)
Train: (4912, 30) \ Test: (890, 30)
Train: (4581, 30) \ Test: (1221, 30)

FOLD 1
----------------------------------------------------------------------------
mean: tensor(51206.5273), std: tensor(638589.6875)
Variables)
	Train:torch.Size([3723, 1, 28])
	Test: torch.Size([2079, 1, 28])
Train Size 3723 Test Size 2079

PATH: ./Model/sparse_fold_1.pt

Starting epoch 1
Loss/ACC after mini-batch   233: 15368.260 / 0.4899
Starting epoch 2
Loss/ACC after mini-batch   233: 11438.299 / 0.5007
Starting epoch 3
Loss/ACC after mini-batch   233: 11925.357 / 0.5060
Starting epoch 4
Loss/ACC after mini-batch   233: 9597.950 / 0.5023
Starting epoch 5
Loss/ACC after mini-batch   233: 7925.050 / 0.5144
Starting epoch 6
Loss/ACC after mini-batch   233: 8651.734 / 0.5120
Starting epoch 7
Loss/ACC after mini-batch   233: 7384.062 / 0.5017
Starting epoch 8
Loss/ACC after mini-batch   233: 7047.151 / 

In [113]:
print(temp[0])
print(temp[1])
print(temp[2])
print(temp[])

tensor(2369.)
tensor(2369., dtype=torch.float64)
tensor(0.5171)


IndexError: list index out of range

In [94]:
'''
            # Compute Prediction Outputs
            preds = outputs.squeeze(1) > 0.0
            

            # Compute loss
            loss = criterion(outputs, targets.unsqueeze(1))

예측값을 위해 아웃풋을 511 -> 51로 만들고
loss를 위해 타겟값을 51 -> 511로 만든다. (아웃풋과 타겟값 매칭): 타겟 기본 크기는 5,1이며 아웃풋 기본 크기는 5,1,1.

뷰를 사용하면 5,1로 맞춰지니, 출력되는 x를 5,1로 맞추면 outputs.squeeze(1)가 필요없어지고 타겟도 건드릴필요가 없음

'''

tsr = temp[0]
print(tsr)
print(f'=> {tsr.size()}\n')
print(tsr.view(tsr.size(0), -1))
print(f'=> {tsr.view(tsr.size(0), -1).size()}\n')
print(f'{tsr.squeeze(1)}\n')
print(f'=> {tsr.squeeze(1).size()}\n')

tensor([[[ 13.6247]],

        [[ 85.3222]],

        [[709.5609]],

        [[ 86.6495]],

        [[ 38.6009]]], grad_fn=<AddBackward0>)
=> torch.Size([5, 1, 1])

tensor([[ 13.6247],
        [ 85.3222],
        [709.5609],
        [ 86.6495],
        [ 38.6009]], grad_fn=<ViewBackward>)
=> torch.Size([5, 1])

tensor([[ 13.6247],
        [ 85.3222],
        [709.5609],
        [ 86.6495],
        [ 38.6009]], grad_fn=<SqueezeBackward1>)

=> torch.Size([5, 1])



In [86]:
tsr = temp[1]
print(f'input size')
print(f'=> {tsr.size()}\n')
# print(tsr.view(tsr.size(0), -1))
print(f'=> {tsr.view(tsr.size(0), -1).size()}\n')

input size
=> torch.Size([5, 1, 28])

=> torch.Size([5, 28])



In [96]:
tsr = temp[2]
print(f'target size')
print(f'=> {tsr.size()}\n')
# print(tsr.view(tsr.size(0), -1))
print(f'=> {tsr.view(tsr.size(0), -1).size()}\n')
print(f'{tsr.unsqueeze(1).size()}')

target size
=> torch.Size([5, 1])

=> torch.Size([5, 1])

torch.Size([5, 1, 1])


In [ ]:
targets.unsqueeze(1)

### Schdule
- 61.36323673186822